# Download Darknet

In [ ]:
!rm -rf darknet
!git clone https://github.com/AlexeyAB/darknet
!pip install gdown

# Setup

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!make

# Download Pre-trained model

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

# Download Experiment Data

In [ ]:
!gdown https://drive.google.com/uc?id=1gnI7zJayZElEpVB-C3niUY07jomLKC-J
!unzip -qo agriculture.zip

# Training Process

In [ ]:
%cd darknet
!./darknet detector train \
                    agriculture/obj.data \
                    agriculture/yolov4-tiny-agriculture.cfg \
                    yolov4-tiny.conv.29 \
                    -dont_show

# Inference

Note: you may need to restart the runtime before running the following codes.

Runtime > Restart runtime

In [ ]:
%cd /content/darknet

## Imports

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import random

import glob
import darknet

from pprint import pprint
import matplotlib.pyplot as plt

## Utilities

In [ ]:
def check_batch_shape(images, batch_size):
    """
        Image sizes should be the same width and height
    """
    shapes = [image.shape for image in images]
    if len(set(shapes)) > 1:
        raise ValueError("Images don't have same shape")
    if len(shapes) > batch_size:
        raise ValueError("Batch size higher than number of images")
    return shapes[0]

def prepare_batch(images, network, channels=3):
    width = darknet.network_width(network)
    height = darknet.network_height(network)

    darknet_images = []
    for image in images:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_resized = cv2.resize(image_rgb, (width, height),
                                   interpolation=cv2.INTER_LINEAR)
        custom_image = image_resized.transpose(2, 0, 1)
        darknet_images.append(custom_image)

    batch_array = np.concatenate(darknet_images, axis=0)
    batch_array = np.ascontiguousarray(batch_array.flat, dtype=np.float32)/255.0
    darknet_images = batch_array.ctypes.data_as(darknet.POINTER(darknet.c_float))
    return darknet.IMAGE(width, height, channels, darknet_images)
    
def batch_detection(network, images, class_names, class_colors,
                    thresh=0.25, hier_thresh=.5, nms=.45, batch_size=4):
    image_height, image_width, _ = check_batch_shape(images, batch_size)
    darknet_images = prepare_batch(images, network)
    batch_detections = darknet.network_predict_batch(network, darknet_images, batch_size, image_width,
                                                     image_height, thresh, hier_thresh, None, 0, 0)
    batch_predictions = []
    for idx in range(batch_size):
        num = batch_detections[idx].num
        detections = batch_detections[idx].dets
        if nms:
            darknet.do_nms_obj(detections, num, len(class_names), nms)
        predictions = darknet.remove_negatives(detections, class_names, num)
        images[idx] = darknet.draw_boxes(predictions, images[idx], class_colors)
        batch_predictions.append(predictions)
    darknet.free_batch_detections(batch_detections, batch_size)
    return images, batch_predictions

def image_detection(image_path, network, class_names, class_colors, thresh):
    # Darknet doesn't accept numpy images.
    # Create one with image we reuse for each detect
    width = darknet.network_width(network)
    height = darknet.network_height(network)
    darknet_image = darknet.make_image(width, height, 3)

    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image_rgb, (width, height),
                               interpolation=cv2.INTER_LINEAR)

    darknet.copy_image_from_bytes(darknet_image, image_resized.tobytes())
    detections = darknet.detect_image(network, class_names, darknet_image, thresh=thresh)
    darknet.free_image(darknet_image)
    image = darknet.draw_boxes(detections, image_resized, class_colors)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB), detections

def visualize(img, predictions, colors):
    output = img.copy()
    print(output.shape)
    for item in predictions:
        label, conf, bbox = item
        cx,cy,w,h = (int(x) for x in bbox)
        x1 = int(cx-w/2)
        x2 = int(cx+w/2)
        y1 = int(cy-h/2)
        y2 = int(cy+h/2)
        output = cv2.rectangle(output, (x1, y1), (x2, y2), colors[label], 2)
      
    output = output[:,:,::-1]
    return output

## Loading the trained model

In [ ]:
batch_size = 16
network, class_names, class_colors = darknet.load_network(
        'agriculture/yolov4-tiny-agriculture.cfg',
        'agriculture/obj.data',
        'agriculture/weights/yolov4-tiny-agriculture_last.weights',
        batch_size=batch_size
)

## Inference

In [ ]:
with open('agriculture/validation.txt') as f:
    image_names = f.readlines()
    image_names = [x.strip() for x in image_names]

random.shuffle(image_names)
image_names = image_names[:batch_size]
images = [cv2.imread(image) for image in image_names]
images, detections,  = batch_detection(network, images, class_names,
                                           class_colors, batch_size=batch_size)


## Visualization

In [ ]:
idx = 15
plt.figure(figsize=(10,10))
plt.imshow(images[idx])

In [ ]:
pprint(detections[1])